In [87]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression  
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import StratifiedKFold


## Exploring data and scaling

In [88]:
df= pd.read_csv('classification.csv')
df.shape

(6362620, 11)

In [89]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [90]:
df['isFraud'].value_counts()/6362620

0    0.998709
1    0.001291
Name: isFraud, dtype: float64

In [91]:
df_sample=df.sample(n=2000000)
df_sample.shape

(2000000, 11)

In [92]:
df_sample.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
4164641,304,CASH_IN,518582.62,C1997888531,326.0,518908.62,C243306413,1235851.99,717269.37,0,0
2241693,186,PAYMENT,37312.36,C1096073367,51377.0,14064.64,M962538020,0.00,0.00,0,0
5275627,372,PAYMENT,9094.54,C223689162,80304.0,71209.46,M303418052,0.00,0.00,0,0
2805590,225,CASH_OUT,194422.54,C266657247,40380.0,0.00,C307867643,23281.97,217704.51,0,0
176313,12,CASH_OUT,416906.47,C1245690599,0.0,0.00,C920740601,942493.16,2668884.40,0,0


#### making sure the sampled data reflects the whole data

In [93]:
df_sample['isFraud'].value_counts()/2000000

0    0.998744
1    0.001257
Name: isFraud, dtype: float64

In [112]:
df_sample.describe()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06,2.000000e+06
mean,2.434416e+02,1.713553e+00,1.800960e+05,9.995343e+05,8.379052e+05,8.591746e+05,3.932684e+05,1.099916e+06,1.224395e+06,1.256500e-03,4.000000e-06
std,1.422229e+02,1.350632e+00,6.035261e+05,5.770904e+05,2.900557e+06,2.936267e+06,2.987344e+05,3.349706e+06,3.624120e+06,3.542488e-02,1.999996e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.000000e+00,1.340960e+04,4.997518e+05,0.000000e+00,0.000000e+00,1.579148e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,1.000000e+00,7.488355e+04,9.995405e+05,1.421465e+04,0.000000e+00,3.150220e+05,1.331082e+05,2.146197e+05,0.000000e+00,0.000000e+00
75%,3.340000e+02,3.000000e+00,2.088598e+05,1.499305e+06,1.074550e+05,1.447966e+05,5.929592e+05,9.434270e+05,1.112279e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,4.000000e+00,7.117248e+07,1.999083e+06,5.039905e+07,4.367380e+07,1.092887e+06,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


#### encoding data

In [94]:
label_encode =preprocessing.LabelEncoder()
df_sample['type']=label_encode.fit_transform(df_sample['type'])
df_sample['nameOrig']=label_encode.fit_transform(df_sample['nameOrig'])
df_sample['nameDest']=label_encode.fit_transform(df_sample['nameDest'])

In [95]:
df_sample.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
4164641,304,0,518582.62,1032114,326.0,518908.62,253700,1235851.99,717269.37,0,0
2241693,186,3,37312.36,98745,51377.0,14064.64,1079547,0.00,0.00,0,0
5275627,372,3,9094.54,1196142,80304.0,71209.46,848781,0.00,0.00,0,0
2805590,225,1,194422.54,1240628,40380.0,0.00,267859,23281.97,217704.51,0,0
176313,12,1,416906.47,253294,0.0,0.00,399996,942493.16,2668884.40,0,0


#### Scaling

In [96]:
scaler = preprocessing.StandardScaler()
scaler.fit(df_sample.drop('isFraud',axis=1))
scaled_features = scaler.transform(df_sample.drop('isFraud',axis=1))
X = pd.DataFrame(scaled_features,columns=df.columns[:-1])
X.rename(columns = {'isFraud':'isFlaggedFraud'}, inplace = True)
Y=df_sample['isFraud']

In [97]:
X

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFlaggedFraud
0,0.425799,-1.268705,0.560849,0.056455,-0.288765,-0.115884,-0.467199,0.040581,-0.139931,-0.002
1,-0.403885,0.952478,-0.236582,-1.560916,-0.271165,-0.287818,2.297288,-0.328362,-0.337846,-0.002
2,0.903922,0.952478,-0.283337,0.340688,-0.261192,-0.268356,1.524809,-0.328362,-0.337846,-0.002
3,-0.129667,-0.528311,0.023738,0.417775,-0.274956,-0.292608,-0.419802,-0.321412,-0.277775,-0.002
4,-1.627316,-0.528311,0.392378,-1.293108,-0.288877,-0.292608,0.022520,-0.046996,0.398577,-0.002
...,...,...,...,...,...,...,...,...,...,...
1999995,0.411737,-1.268705,0.267836,0.124718,9.790738,9.780809,-0.504918,2.779401,2.440305,-0.002
1999996,0.728141,-1.268705,-0.284706,-1.080738,3.298717,3.254171,-0.861161,-0.302666,-0.316377,-0.002
1999997,0.939078,-0.528311,-0.098364,0.538472,-0.288877,-0.292608,-0.538510,-0.104104,-0.097256,-0.002
1999998,-1.381224,-0.528311,-0.152751,1.104310,-0.288877,-0.292608,-0.273642,-0.202545,-0.197299,-0.002


In [98]:
Y

4164641    0
2241693    0
5275627    0
2805590    0
176313     0
          ..
4107828    0
4827012    0
5406087    0
1021207    0
2442028    0
Name: isFraud, Length: 2000000, dtype: int64

#### splitting data into 60% train, 20% test and 20% cross validation

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X,Y
                                                    ,test_size=0.40 )
X_cv,X_test1,y_cv,y_test1=train_test_split(X_test,y_test
                                                    ,test_size=0.50)

In [100]:
X_train.shape,y_train.shape

((1200000, 10), (1200000,))

In [101]:
X_test1.shape,y_test1.shape

((400000, 10), (400000,))

In [102]:
X_cv.shape,y_cv.shape

((400000, 10), (400000,))

## logistic regression model

In [103]:
model=linear_model.LogisticRegression(penalty='none')
model.fit(X_train,y_train)
Y_predict_test1=model.predict(X_test1)
Y_predict_cv=model.predict(X_cv)
r=mean_squared_error(y_test1, Y_predict_test1)
v=mean_squared_error(y_cv, Y_predict_cv)  
c=accuracy_score(y_test1,Y_predict_test1) 
d=accuracy_score(y_cv, Y_predict_cv)                      
print('MSE for test data= '+str(r))
print('MSE for cross validation data= '+str(v))
print('accuracy for test data= '+str(c)) 
print('accuracy for cross validation data= '+str(d))                     

MSE for test data= 0.000675
MSE for cross validation data= 0.000685
accuracy for test data= 0.999325
accuracy for cross validation data= 0.999315


In [104]:
conf=confusion_matrix(y_test1,Y_predict_test1)
conf

array([[399483,     30],
       [   240,    247]])

# K-fold and stratified k-fold

In [105]:
cv = KFold(n_splits=20, random_state=0, shuffle=True)
scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.999 (0.000)


In [106]:
cv_stratified= StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
scores_stratified = cross_val_score(model, X, Y, scoring='accuracy', cv=cv_stratified, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores_stratified), std(scores_stratified)))

Accuracy: 0.999 (0.000)


# Regularization

#### Reige

In [107]:
lamda=[0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
listoferror=[]
for i in lamda:
    lr_reg=LogisticRegression(C=1/i)
    lr_reg.fit(X_train,y_train)
    pred_val_reg=lr_reg.predict(X_cv)
    accuracy_val=accuracy_score(y_cv,pred_val_reg)
    err_cv=1-accuracy_val
    listoferror.append(1-accuracy_val)
listoferror


/Users/AliElsaidy/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/AliElsaidy/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

[0.0007175000000000376,
 0.0007175000000000376,
 0.0007224999999999593,
 0.0007350000000000412,
 0.000747500000000012,
 0.0007650000000000157,
 0.0007724999999999538,
 0.0008049999999999446,
 0.0008424999999999683,
 0.0008799999999999919,
 0.0009350000000000191]

In [108]:
lamda[np.argmin(listoferror)]

0.01

#### Lasso

In [109]:
lamda1=[0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
listoferror1=[]
for i in lamda1:
    lr_reg=LogisticRegression(penalty='l1', C=i, solver='saga')
    lr_reg.fit(X_train,y_train)
    pred_val_reg=lr_reg.predict(X_cv)
    accuracy_val=accuracy_score(y_cv,pred_val_reg)
    err_cv=1-accuracy_val
    listoferror1.append(1-accuracy_val)
listoferror1

/Users/AliElsaidy/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/AliElsaidy/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/AliElsaidy/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/AliElsaidy/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/AliElsaidy/opt/anaconda3/lib/python3.8/site-packages/skle

[0.001227500000000048,
 0.001214999999999966,
 0.0012124999999999497,
 0.0012100000000000444,
 0.0012100000000000444,
 0.0012100000000000444,
 0.0012100000000000444,
 0.0012100000000000444,
 0.0012100000000000444,
 0.0012100000000000444,
 0.0012100000000000444]

In [110]:
lamda1[np.argmin(listoferror1)]

0.08